https://www.youtube.com/watch?v=xdq6Gz33khQ&list=TLPQMTMwMTIwMjJO8mOZGdqcXA&index=4

In [1]:
!pip install requests

In [1]:
import requests
import datetime
from urllib.parse import urlencode

import base64

In [2]:
client_id = '8e6e962237b74490aca58fd858c5a54f'
client_secret = '06fd25cc7b164f7e88927938c5c1a589'

In [6]:
# This is now a reusable Spotify API object

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist'): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()}) #type has to be lower!
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
spotify.search("Pura droga sin cortar", search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Pura+droga+sin+cortar&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1DH9RJ0xBVje6gQmK3LWUY'},
       'href': 'https://api.spotify.com/v1/artists/1DH9RJ0xBVje6gQmK3LWUY',
       'id': '1DH9RJ0xBVje6gQmK3LWUY',
       'name': 'Violadores Del Verso',
       'type': 'artist',
       'uri': 'spotify:artist:1DH9RJ0xBVje6gQmK3LWUY'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ

In [7]:
spotify.get_artist('1DH9RJ0xBVje6gQmK3LWUY')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1DH9RJ0xBVje6gQmK3LWUY'},
 'followers': {'href': None, 'total': 453099},
 'genres': ['boom bap espanol',
  'rap underground espanol',
  'spanish hip hop',
  'venezuelan hip hop'],
 'href': 'https://api.spotify.com/v1/artists/1DH9RJ0xBVje6gQmK3LWUY',
 'id': '1DH9RJ0xBVje6gQmK3LWUY',
 'images': [{'height': 665,
   'url': 'https://i.scdn.co/image/cf578e36ae2ac2fd006c4945c4e40f06029f3eda',
   'width': 1000},
  {'height': 426,
   'url': 'https://i.scdn.co/image/0c3dd9c763d9b2bb3094b34fd09740d697bcebff',
   'width': 640},
  {'height': 133,
   'url': 'https://i.scdn.co/image/2525224ea61479c27d5bf4b425770cb020769532',
   'width': 200},
  {'height': 43,
   'url': 'https://i.scdn.co/image/1d8826b16be942c25898f6b276335268fc11746c',
   'width': 64}],
 'name': 'Violadores Del Verso',
 'popularity': 56,
 'type': 'artist',
 'uri': 'spotify:artist:1DH9RJ0xBVje6gQmK3LWUY'}